# I. Setup Up
   ### I.1 Assigning Paths

In [1]:
workspace = 'Tensorflow/workspace'
scripts = 'Tensorflow/scripts'
apiModel = 'Tensorflow/models'
annotations = workspace+'/annotations'
images = workspace+'/images'
model = workspace+'/models'
preTrainedModel = workspace+'/pre-trained-models'
configPipeline = model+'/my_ssd_mobnet/pipeline.config'
newModelName = 'my_ssd_mobnet' 
checkpoint = model+'/my_ssd_mobnet/'

### I.2 Creating Our Label Map

In [2]:
#Defining the signs we are using for this project
labels = [{'name':'Hi', 'id':1}, 
          {'name':'I Love You', 'id':2},
          {'name':'Thank You', 'id':3}, 
          {'name':'Yes', 'id':4},
          {'name':'No', 'id':5}]

#Creating the label_map file
with open(annotations + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### I.3 Generating TF records

In [2]:
!python {scripts + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {annotations + '/label_map.pbtxt'} -o {annotations + '/train.record'}
!python {scripts + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {annotations + '/label_map.pbtxt'} -o {annotations + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# II. Training Using Transfer Learning
### II.1 Get Tensorflow Models

In [26]:
#!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  55% (1435/2587)
Updating files:  56% (1449/2587)
Updating files:  57% (1475/2587)
Updating files:  58% (1501/2587)
Updating files:  59% (1527/2587)
Updating files:  60% (1553/2587)
Updating files:  61% (1579/2587)
Updating files:  62% (1604/2587)
Updating files:  63% (1630/2587)
Updating files:  64% (1656/2587)
Updating files:  65% (1682/2587)
Updating files:  66% (1708/2587)
Updating files:  67% (1734/2587)
Updating files:  68% (1760/2587)
Updating files:  69% (1786/2587)
Updating files:  70% (1811/2587)
Updating files:  71% (1837/2587)
Updating files:  72% (1863/2587)
Updating files:  73% (1889/2587)
Updating files:  74% (1915/2587)
Updating files:  75% (1941/2587)
Updating files:  76% (1967/2587)
Updating files:  77% (1992/2587)
Updating files:  78% (2018/2587)
Updating files:  79% (2044/2587)
Updating files:  80% (2070/2587)
Updating files:  81% (2096/2587)
Updating files:  82% (2122/2587)
Updating files:  83% (2148/2587)
Updating files:  8

### II.2 Copy Model Config to Training Folder

In [ ]:
'''
We simply create a new folder under 'tensorflow/workspace/models/' named after our new model.
Inside that directory we copy the pre-trained ssd_mobilenet_v2_coco model's config.
!mkdir {'Tensorflow\workspace\models\\'+newModelName}
'''

### II.3 Customizing Our Config For Transfer Learning

In [2]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [3]:
config = config_util.get_configs_from_pipeline_file(configPipeline)

In [4]:
config

{'model': ssd {
   num_classes: 5
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [5]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(configPipeline, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

pipeline_config.model.ssd.num_classes = 5 #We have 5 classes/Signs
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = preTrainedModel+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= annotations + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [annotations + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = annotations + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [annotations + '/test.record']

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(configPipeline, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

### II.4 Training

In [34]:
#cmd to use for our model's training
#We might want to consider a higher number of steps for better results
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(apiModel, model,newModelName,model,newModelName))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# III. Testing Detection
### III.1 Loading Our Model 

In [6]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [7]:
# Load the pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(configPipeline)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpoint, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

### III.2 Testing Real-Time Detection

In [ ]:
#Notes for troubleshooting:

'''
-There might be a chance the camera is used used by another process
- captured frame might be empty: print(input_tensor.shape)
-cap.release()
- The ID for the camera might sometimes change depending on the process and the device on which the project is executed
, hence consider changing it
-type(detections)
-Opencv version 4.5.3 is not recommended!
import cv2
print(cv2.__version__)

'''

In [8]:
import cv2 
import numpy as np
#import matplotlib.pyplot as plt

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(annotations+'/label_map.pbtxt')

In [10]:
# Prepare Our Capture
capture = cv2.VideoCapture(0)
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = capture.read()
    image_np = np.array(frame)
    print(image_np.shape)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    print(input_tensor.shape)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        capture.release()
        break

In [ ]:
detections = detect_fn(input_tensor)

print(input_tensor.shape)


# IV. Set Up for App Creation

### Saving Graph and Coverting Into TFJS Model

In [5]:
#cmd to export our checkpoints
print(""" python {}/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path={}/{}/pipeline.config --trained_checkpoint_dir={} --output_directory={}exp
""".format(apiModel,model,newModelName,checkpoint,checkpoint))

 python Tensorflow/models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/my_ssd_mobnet/ --output_directory=Tensorflow/workspace/models/my_ssd_mobnet/exp



In [6]:
!pip install tensorflowjs

  Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0


ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement six~=1.15.0, but you'll have six 1.12.0 which is incompatible.
ERROR: apache-beam 2.31.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.2 which is incompatible.


In [11]:
"""tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {}exp/saved_model {}conv""".format(checkpoint,checkpoint)

"tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Tensorflow/workspace/models/my_ssd_mobnet/exp/saved_model Tensorflow/workspace/models/my_ssd_mobnet/conv"